In [2]:
def dense_block(x, blocks, growth_rate, name):

    for i in range(blocks):
        x = conv_block(x, growth_rate, name=name + '_block' + str(i + 1))
    return x

def conv_block(x, growth_rate, name):
    bn_axis = 2
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
    x1 = layers.Conv1D(4 * growth_rate, 5, use_bias=True, padding='same', kernel_initializer='he_normal', name=name + '_1_conv')(x1)

    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv1D(grow th_rate, 10, padding='same', use_bias=True, kernel_initializer='he_normal', name=name + '_2_conv')(x1)

    x = layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])

    # x = layers.Conv1D(growth_rate, 1, padding='same', use_bias=False, name=name + '_3_conv')(x)
    # x = layers.Add(name=name + '_concat')([x, x1])

    return x



def transition_block(x, reduction, name):
    bn_axis = 2
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    x = layers.Conv1D(int(backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False, kernel_initializer='he_normal', name=name + '_conv')(x)
                      
#     x = layers.AveragePooling1D(2, strides=2, name=name + '_pool')(x)
    x = layers.MaxPooling1D(2, strides=2, name=name + '_pool')(x)

    # x = spatial_pyramid_pool(x, 3, bn_axis, name)
        
    return x


SyntaxError: invalid syntax (<ipython-input-2-68a32e73cb40>, line 15)

In [3]:
from keras.engine.topology import Layer
import keras.backend as K


class SpatialPyramidPooling(Layer):
    """Spatial pyramid pooling layer for 2D inputs.
    See Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition,
    K. He, X. Zhang, S. Ren, J. Sun
    # Arguments
        pool_list: list of int
            List of pooling regions to use. The length of the list is the number of pooling regions,
            each int in the list is the number of regions in that pool. For example [1,2,4] would be 3
            regions with 1, 2x2 and 4x4 max pools, so 21 outputs per feature map
    # Input shape
        4D tensor with shape:
        `(samples, channels, rows, cols)` if dim_ordering='th'
        or 4D tensor with shape:
        `(samples, rows, cols, channels)` if dim_ordering='tf'.
    # Output shape
        2D tensor with shape:
        `(samples, channels * sum([i * i for i in pool_list])`
    """

    def __init__(self, pool_list, **kwargs):

        self.pool_list = pool_list

        self.num_outputs_per_channel = sum([i for i in pool_list])

        super(SpatialPyramidPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.nb_channels = input_shape[2]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.nb_channels * self.num_outputs_per_channel)

    def get_config(self):
        config = {'pool_list': self.pool_list}
        base_config = super(SpatialPyramidPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, x, mask=None):

        input_shape = K.shape(x)

        num_rows = input_shape[1]
        # num_cols = input_shape[2]

        row_length = [K.cast(num_rows, 'float32') / i for i in self.pool_list]
        # col_length = [K.cast(num_cols, 'float32') / i for i in self.pool_list]


        outputs = []


        for pool_num, num_pool_regions in enumerate(self.pool_list):
            for jy in range(num_pool_regions):
                # for ix in range(num_pool_regions):
                #     x1 = ix * col_length[pool_num]
                #     x2 = ix * col_length[pool_num] + col_length[pool_num]
                    y1 = jy * row_length[pool_num]
                    y2 = jy * row_length[pool_num] + row_length[pool_num]

                    # x1 = K.cast(K.round(x1), 'int32')
                    # x2 = K.cast(K.round(x2), 'int32')
                    y1 = K.cast(K.round(y1), 'int32')
                    y2 = K.cast(K.round(y2), 'int32')

                    # new_shape = [input_shape[0], y2 - y1,
                    #                 x2 - x1, input_shape[3]]

                    new_shape = [input_shape[0], y2 - y1, input_shape[2]]


                    x_crop = x[:, y1:y2, :]
                    xm = K.reshape(x_crop, new_shape)
                    pooled_val = K.max(xm, axis=1)
#                     pooled_val = K.mean(xm, axis=1)
                    outputs.append(pooled_val)

        #outputs = K.concatenate(outputs,axis = 1)
        outputs = K.concatenate(outputs)
        #outputs = K.reshape(outputs,(len(self.pool_list),self.num_outputs_per_channel,input_shape[0],input_shape[1]))
        #outputs = K.permute_dimensions(outputs,(3,1,0,2))
        #outputs = K.reshape(outputs,(input_shape[0], self.num_outputs_per_channel * self.nb_channels))

        return outputs

Using TensorFlow backend.


In [4]:
import math

def cropping(K=4, extension_rate=4):
    def crop(x):
        crop_list = list()
        input_shape = x.shape[-2].value
        _k_pool = [2**x for x in range(K)]
        _k_list = [int(input_shape*(0.5**x)) for x in range(K)]

#         print(input_shape)
#         print(_k_pool)
#         print(_k_list)

        for ind, value in enumerate(_k_pool):
            for ii in range(value):
                y1 = ii * _k_list[ind]
                y2 = ii * _k_list[ind] + _k_list[ind]
    
#                 print(y1,y2)
                
#                 padding1 = math.ceil((input_shape-_k_list[ind])*0.5)
#                 padding2 = math.floor((input_shape-_k_list[ind])*0.5)
                
                padding1 = abs(0-y1)
                padding2 = abs(input_shape-y2)
                
#                 print(padding1,padding2)

                croped = layers.Lambda( lambda x: x[:,y1:y2,:])(x)
                
                croped_pad = layers.ZeroPadding1D(padding =(padding1, padding2))(croped)
                
                y = layers.Conv1D(extension_rate, 5, padding='same', use_bias=True, kernel_initializer='he_normal')(croped_pad)
                
                crop_list.append(y)
                
        y = layers.Concatenate(axis=-1)(crop_list)
        return y
    return crop

In [5]:
def DCNNNetNew(extension_rate = None,
            input_shape = None,
            classes = None,
            **kwargs):
    

    inputs = layers.Input(shape=input_shape, name='input')

    x = cropping(K=4, extension_rate=10)(inputs)

    blocks = [4,4,4,4]

    x = dense_block(x, blocks[0], extension_rate, name='conv2')
    x = transition_block(x, 0.5, name='pool2')

    x = dense_block(x, blocks[1], extension_rate, name='conv3')
    x = transition_block(x, 0.5, name='pool3')

    x = dense_block(x, blocks[2], extension_rate, name='conv4')
    x = transition_block(x, 0.5, name='pool4')

    x = dense_block(x, blocks[3], extension_rate, name='conv5')
    x = transition_block(x, 0.5, name='pool5')
    
#     x = dense_block(x, blocks[4], extension_rate, name='conv6')
#     x = transition_block(x, 0.5, name='pool6')
    
#     x = SpatialPyramidPooling([1,2,4])(x)
    
    x = layers.GlobalAveragePooling1D(name='avg_pool')(x)
    x = layers.Dense(classes, activation='softmax', name='fc')(x)

    model = models.Model(inputs, x, name=f'DCNNNet_concate_new')

    return model





In [6]:

def createModelNew(input_shape=(100,1), extension_rate=12, classes=3 ):
#     inputs = layers.Input(shape=input_shape)

    model = DCNNNetNew(extension_rate=extension_rate, input_shape=input_shape, classes=classes)
    model.summary()
    model.name
    return model